In [1]:
load('https://raw.githubusercontent.com/cerickson30/qBound/main/spectator_floor_display_functions.py')

In [2]:
print('Importing full uspcm_dict...')

full_uspcm_dict = get_full_uspcm_dict()
    
print('Full uspcm_dict has been imported.')

Importing full uspcm_dict...
Full uspcm_dict has been imported.


In [3]:
%%cython # use cython to speed up computations -- This jupyter magic call cannot be in a .py script file

# use garbage collection to manage memory usage
import gc
gc.enable()


def usp_comp(amat):
    """ Return the spectator number of the graph, which is defined to be
    the size of the smallest unique-shortest-path complement of a graph.
    
    :param amat: A graph or adjacency matrix of a graph.
    """
    # Accept graph or matrix input
    try:
        # Get size from the number of columns, if amat is a matrix
        nn = amat.ncols()
    except AttributeError:
        # If amat is a graph or graph6_string, convert to adjacency matrix
        amat = amat.adjacency_matrix()
        # Get size from number of colummns
        nn = amat.ncols()
    if not amat:
        return nn - 1
    
    # Use fact that (i,j)-entry of A^k is the number of i-j walks of length k
    # to determine the length of the longest unique shortest path (the largest k
    # for which there exists an (i,j)-entry of A^k equal to 1.)
    A = amat + 2
    AA = A + 0
    compsize = nn - 1
    while min(min([yy for yy in xx if yy]) for xx in AA) == 1:
        compsize -= 1
        AA = AA*A
    
    return compsize



def Glabel(G):
    """
    Returns the graph6_string of the canonical labeling of graph G using the sage algorithm
    to determine the canonical labeling.
    
    :param G: A graph object.
    """
    return G.canonical_label(algorithm='sage').graph6_string()


def edgeclasses(G):
    """
    Generator function to generate the automorphism groups of the edges in graph G.
    
    :param G: A graph object.
    """
    Aut = G.automorphism_group()
    needs = {(xx[0], xx[1]): True for xx in G.edges()}
    while needs:
        anedge = next(iter(needs))
        yield anedge
        for xx in Aut.orbit(anedge, action='OnPairs'):
            if (xx[0], xx[1]) in needs:
                del needs[(xx[0], xx[1])]
            if (xx[1], xx[0]) in needs:
                del needs[(xx[1], xx[0])]

                
def deletions(G):
    """
    Generator function to generate the minors of G which are achievable by deleting a single edge
    from G.
    Returns the graph6_string of the canonical labeling.
    
    :param G: A graph object.
    """
    for ed in edgeclasses(G):
        H = G.copy()
        H.delete_edge(ed)
        yield Glabel(H)

        
def contractions(G):
    """
    Generator function to generate the minors of G which are achievable by either contracting a single
    edge in G or deleting a single isolated vertex in G.
    Returns the graph6_string of the canonical labeling.
    
    :param G: A graph object.
    """
    for ed in edgeclasses(G):
        H = G.copy()
        H.contract_edge(ed)
        yield Glabel(H)
    # Also covers isolated vertex deletion
    if 0 in G.degree():
        H = G.copy()
        H.delete_vertex(G.degree().index(0))
        yield Glabel(H)

ld: warning: -pie being ignored. It is only used when linking a main executable


In [4]:
def progressBar(iterable, prefix = '', suffix = '', decimals = 1, length = 100, fill = '█', printEnd = "\r"):
    import datetime
    """
    Call in a loop to create terminal progress bar
    @params:
        iteration   - Required  : current iteration (Int)
        total       - Required  : total iterations (Int)
        prefix      - Optional  : prefix string (Str)
        suffix      - Optional  : suffix string (Str)
        decimals    - Optional  : positive number of decimals in percent complete (Int)
        length      - Optional  : character length of bar (Int)
        fill        - Optional  : bar fill character (Str)
        printEnd    - Optional  : end character (e.g. "\r", "\r\n") (Str)
    """
    total = len(iterable)
    # Progress Bar Printing Function
    def printProgressBar(iteration):
        try:
            percent = ("{0:." + str(decimals) + "f}").format(100 * (iteration / float(total)))
            filledLength = int(length * iteration // total)
        except ZeroDivisionError:
            percent = 100
            filledLength = length
        
        bar = fill * filledLength + '-' * (length - filledLength)
        print(f'\r{datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")} {prefix} |{bar}| {percent}% {suffix}', end = printEnd, flush=False)
    # Initial Call
    printProgressBar(0)
    # Update Progress Bar
    for i, item in enumerate(iterable):
        yield item
        printProgressBar(i + 1)
    # Print New Line on Complete
    print()

In [5]:
full_uspcm_dict.keys()

dict_keys(['0_verts', '1_verts', '2_verts', '3_verts', '4_verts', '5_verts', '6_verts', '7_verts', '8_verts', '9_verts', '10_verts'])

In [6]:
full_uspcm_dict.get('9_verts').keys()

dict_keys(['36_edges', '35_edges', '34_edges', '33_edges', '32_edges', '31_edges', '30_edges', '29_edges', '28_edges', '27_edges', '26_edges', '25_edges', '24_edges', '23_edges', '22_edges', '21_edges', '20_edges', '19_edges', '18_edges', '17_edges', '16_edges', '15_edges', '14_edges', '13_edges', '12_edges', '11_edges', '10_edges', '9_edges', '8_edges', '7_edges', '6_edges', '5_edges', '4_edges', '3_edges', '2_edges', '1_edges'])

In [7]:
spread_dict = {}
large_spread = {f'spread_{s}':[] for s in range(2,10)}

In [8]:
for nn in range(2, 11):
    for ee in range(nn*(nn-1)//2, 1, -1):
        for g6_str in full_uspcm_dict.get(f'{nn}_verts').get(f'{ee}_edges'):

            spec_floor = full_uspcm_dict.get(f'{nn}_verts').get(f'{ee}_edges').get(g6_str)

            G = Graph(g6_str)
            spread = usp_comp(G) - spec_floor
            if spread < 0:
                print('UhOh')
                break
            
            if spread > 0:
                spread_dict[g6_str] = spread

            if spread > 1:
                large_spread[f'spread_{spread}'].append(g6_str)

KeyboardInterrupt: 

In [9]:
for spread in large_spread:
    length = len(large_spread[spread])
    print(f'Number of graphs whose spread is {spread[-1]}: {length}')

Number of graphs whose spread is 2: 2080
Number of graphs whose spread is 3: 212
Number of graphs whose spread is 4: 7
Number of graphs whose spread is 5: 0
Number of graphs whose spread is 6: 0
Number of graphs whose spread is 7: 0
Number of graphs whose spread is 8: 0
Number of graphs whose spread is 9: 0
